In [ ]:
from collections import namedtuple
import random

In [ ]:
import pandas as pd

In [ ]:
import orchid
import orchid.reference_origins as origins

In [ ]:
import orchid.project_loader as opl

In [ ]:
import toolz.curried as toolz

In [ ]:
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
import UnitsNet

In [ ]:
# Path name of whatever project you wish to load.
loader = opl.ProjectLoader(
    r'c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.ifrac')

In [ ]:
# The returned value from `native_project` is . NET project. NOTE: each instance 
# of the `ProjectLoader` class loads **exactly one** .NET project. To load a 
# different project, you must create a new `ProjectLoader` instance.
native_project = loader.native_project()

In [ ]:
all_wells = native_project.Wells.Items

In [ ]:
well_of_interest = all_wells[3]  # 'Demo_4H'

In [ ]:
well_of_interest.Name

In [ ]:
all_stages_for_well_of_interest = well_of_interest.Stages.Items

In [ ]:
len(all_stages_for_well_of_interest)

In [ ]:
all_stage_numbers = list(range((len(all_stages_for_well_of_interest))))
random.shuffle(all_stage_numbers)
all_stage_numbers

In [ ]:
sampled_stage_numbers = [0, 34, 4, 28]

In [ ]:
@toolz.curry
def basic_data_for_stage_of_well(w, s):
    return ('bakken', w.Name, s.DisplayStageNumber,
            s.DisplayNameWithoutWell, s.OrderOfCompletionOnWell,
            s.GlobalStageSequenceNumber, stage_type_str(s.StageType))
basic_data_for_well = basic_data_for_stage_of_well(well_of_interest)

@toolz.curry
def stage_type_str(stage_type):
    stage_type_str_map = {0: 'PlugAndPerf',
                          1: 'SlidingSleeve',
                          2: 'SinglePointEntry',
                          3: 'OpenHole'}
    return stage_type_str_map[stage_type]

@toolz.curry
def make_basic_item(n):
    basic_data = basic_data_for_well(all_stages_for_well_of_interest[n])
    return {
        'field': [basic_data[0]],
        'name': [basic_data[1]],
        'stage_no': [basic_data[2]],
        'name_without_well': [basic_data[3]],
        'order': [basic_data[4]],
        'global_seq_no': [basic_data[5]],
        'stage_type': [basic_data[6]],
    }
    
all_items = toolz.pipe(sampled_stage_numbers,
                       toolz.map(make_basic_item),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)

In [ ]:
all_frames = [rf for rf in iter(origins.WellReferenceFrameXy)]
random_frames = list(toolz.concat([all_frames] * 2))
random.shuffle(random_frames)
random_frames

In [ ]:
sampled_frames = [
    origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
    origins.WellReferenceFrameXy.WELL_HEAD,
    origins.WellReferenceFrameXy.PROJECT,
    origins.WellReferenceFrameXy.WELL_HEAD,
]

In [ ]:
SubsurfacePoint = namedtuple('SubsurfacePoint', ['x', 'y', 'depth'])
@toolz.curry
def subsurface_location_in_units(in_units, subsurface_location):
    x = subsurface_location.X.ToUnit(in_units)
    y = subsurface_location.Y.ToUnit(in_units)
    depth = subsurface_location.Depth.ToUnit(in_units)
    return SubsurfacePoint(x, y, depth)
subsurface_location_in_project_units = subsurface_location_in_units(native_project.ProjectUnits.LengthUnit)

def get_stage_bottom_location(reference_frame, stage):
    return stage.GetStageLocationBottom(reference_frame.value, DepthDatum.KellyBushing)

def get_stage_top_location(reference_frame, stage):
    return stage.GetStageLocationTop(reference_frame.value, DepthDatum.KellyBushing)

@toolz.curry
def stage_location(w, location_func, rf, s):
    subsurface_location = location_func(rf, s)
    x, y, depth = subsurface_location_in_project_units(subsurface_location)
    return ('bakken', w.Name, s.DisplayStageNumber, rf.name,
            f'{x.Value:.2f} {UnitsNet.Length.GetAbbreviation(x.Unit)}',
            f'{y.Value:.2f} {UnitsNet.Length.GetAbbreviation(y.Unit)}',
            f'{depth.Value:.2f} {UnitsNet.Length.GetAbbreviation(depth.Unit)}')
location_for_well = stage_location(well_of_interest)
stage_bottom_location_for_well = location_for_well(get_stage_bottom_location)
stage_top_location_for_well = location_for_well(get_stage_top_location)

@toolz.curry
def make_location_item(rf, n):
    # location_data = stage_top_location_for_well(rf, all_stages_for_well_of_interest[n])
    location_data = stage_bottom_location_for_well(rf, all_stages_for_well_of_interest[n])
    return {
        'field': [location_data[0]],
        'name': [location_data[1]],
        'stage_no': [location_data[2]],
        'frame': [location_data[3]],
        'x': [location_data[4]],
        'y': [location_data[5]],
        'depth': [location_data[6]],
    }

all_items = toolz.pipe(zip(sampled_frames,
                           sampled_stage_numbers,),
                       toolz.map(lambda pair: make_location_item(*pair)),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)

In [ ]:
@toolz.curry
def cluster_count_for_stage_of_well(w, s):
    return ('bakken', w.Name, s.DisplayStageNumber,
            s.NumberOfClusters)
cluster_count_for_well = cluster_count_for_stage_of_well(well_of_interest)

@toolz.curry
def make_cluster_count(n):
    cluster_count_data = cluster_count_for_well(all_stages_for_well_of_interest[n])
    return {
        'field': [cluster_count_data[0]],
        'name': [cluster_count_data[1]],
        'stage_no': [cluster_count_data[2]],
        'cluster_count': [cluster_count_data[3]],
    }

all_items = toolz.pipe(sampled_stage_numbers,
                       toolz.map(make_cluster_count),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

cluster_count_df = pd.DataFrame(data)
cluster_count_df

In [ ]:
shuffled_cluster_numbers = []
for cluster_count in cluster_count_df['cluster_count']:
    candidates = list(range(1, cluster_count + 1))
    random.shuffle(candidates)
    shuffled_cluster_numbers.append(candidates)
shuffled_cluster_numbers

In [ ]:
sampled_cluster_numbers = [4, 6, 1, 5]

In [ ]:
def get_stage_cluster_location(reference_frame, stage, cluster_no):
    return stage.GetStageLocationCluster(cluster_no, reference_frame.value, DepthDatum.KellyBushing)

@toolz.curry
def stage_cluster_location(w, location_func, rf, s, cluster_no):
    subsurface_location = location_func(rf, s, cluster_no)
    x, y, depth = subsurface_location_in_project_units(subsurface_location)
    return ('bakken', w.Name, s.DisplayStageNumber, cluster_no, rf.name,
            f'{x.Value:.2f} {UnitsNet.Length.GetAbbreviation(x.Unit)}',
            f'{y.Value:.2f} {UnitsNet.Length.GetAbbreviation(y.Unit)}',
            f'{depth.Value:.2f} {UnitsNet.Length.GetAbbreviation(depth.Unit)}')
cluster_location_for_well = stage_cluster_location(well_of_interest)
stage_cluster_location_for_well = cluster_location_for_well(get_stage_cluster_location)

@toolz.curry
def make_cluster_location_item(rf, n, cluster_no):
    location_data = stage_cluster_location_for_well(rf, all_stages_for_well_of_interest[n], cluster_no)
    return {
        'field': [location_data[0]],
        'name': [location_data[1]],
        'stage_no': [location_data[2]],
        'cluster_no': [location_data[3]],
        'frame': [location_data[4]],
        'x': [location_data[5]],
        'y': [location_data[6]],
        'depth': [location_data[7]],
    }

all_items = toolz.pipe(zip(sampled_frames,
                           sampled_stage_numbers,
                           sampled_cluster_numbers,),
                       toolz.map(lambda triple: make_cluster_location_item(*triple)),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)